In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
## Simulation to show the proof fails :(

import numpy as np
def log_no(q, ps, ns):
    return -np.sum(ns * np.log(1 - q * ps))
for i in range(5000):
    N = 2#np.random.randint(low=100, high=2000)
    ps = np.random.uniform(size=N)
    n_a = np.random.randint(low=0, high=1000, size=N)
    n_b = np.random.randint(low=0, high=1000, size=N)
    q0 = np.random.uniform(low=0, high=1e-3,size=1)[0]
    no_a0 = log_no(q0, ps, n_a)
    no_b0 = log_no(q0, ps, n_b)
    order0 = (no_a0 > no_b0)
    if order0:
        err_msg = f'{i}: q0 = {q0}, no_a0 = {no_a0} > {no_b0} = no_b0\n'
    else:
        err_msg = f'{i}: q0 = {q0}, no_b0 = {no_b0} > {no_a0} = no_a0\n'

    for q in np.random.uniform(low=0, high=q0,size=1000):
        no_a = log_no(q, ps, n_a)
        no_b = log_no(q, ps, n_b)
        order = (no_a > no_b)       
        if order0 != order:
            if order:
                err_msg = err_msg + f'q = {q}, no_a = {no_a} > {no_b} = no_b\n'
            else:
                err_msg = err_msg + f'q = {q}, no_b = {no_b} > {no_a} = no_a\n'
            print(err_msg)
            
            assert False
            
            



In [43]:
pd.read_csv('./tables/TableS1.csv')

,Origin,risk_i,municipality,name,region,continent
0,PEK,0.740752,Beijing,Beijing Capital International Airport,CN-11,AS
1,HKG,0.626858,Hong Kong,Hong Kong International Airport Kai Tak,HK-U-A,AS
2,SIN,0.526148,Singapore,Singapore Changi Airport,SG-04,AS
3,DEL,0.496644,New Delhi,Indira Gandhi International Airport,IN-DL,AS
4,DXB,0.491557,Dubai,Dubai International Airport,AE-DU,AS
...,...,...,...,...,...,...
1359,LRD,0.000000,Laredo,Laredo International Airport,US-TX,NaN
1360,LFT,0.000000,Lafayette,Lafayette Regional Airport,US-LA,NaN
1361,BYM,0.000000,Bayamo,Carlos Manuel de Cespedes Airport,CU-12,NaN
1362,GAO,0.000000,Guantánamo,Mariana Grajales Airport,CU-14,NaN


In [27]:
import math
import numpy as np
def get_p2(p1, n1, n2, q=1e-6):
    enum = np.power( (1 - q*(1-p1)), n2/n1) - 1
    return 1 + enum / q

def f(p, n, q):
    return (1 - q * (1-p))**n

p1 = 0.01
n1 = 10000
n2 = 20009
q = 1e-6
p2 = get_p2(p1=p1, n1=n1, n2=n2, q=q)

for q in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8]:
    print(f(p1, n1, q) - f(p2, n2, q))

0.3525706212972674
0.23297794801271188
0.029009289218990708
0.0029662085213627565
0.00029728264841577534
